# Learning and Decision Making

## Laboratory 4: Partially observable Markov decision problems

In the end of the lab, you should submit all code/answers written in the tasks marked as "Activity n. XXX", together with the corresponding outputs and any replies to specific questions posed to the e-mail <adi.tecnico@gmail.com>. Make sure that the subject is of the form [&lt;group n.&gt;] LAB &lt;lab n.&gt;.

### 1. Modeling

Consider once again the guessing game domain described in the Homework and which you described as a POMDP.

Recall that:

* The opponent can hold one of two cards in hand: an Ace of Clubs (A&clubs;) and an Ace of Diamonds (A&diams;). The agent must guess which card the opponent is holding. 

* For every right answer, the agent wins 1EUR, and every wrong answer costs the agent 1EUR. 

* The agent can also try to _peek_. 

* When the agent peeks, it sees the right card with a probability of 0.9 and the wrong card with probability 0.1.

* The game restarts whenever the agent makes a guess.

Consider throughout that $\gamma=0.9$.

---

#### Activity 1.        

Implement your POMDP in Python. In particular,

* Create a list with all the states;
* Create a list with all the actions;
* Create a list with all the observations
* For each action, define a `numpy` array with the corresponding transition probabilities;
* For each action, define a `numpy` array with the corresponding observation probabilities;
* Define a `numpy`array with the cost that you defined in your homework.

The order for the states and actions used in the transition probability and cost matrices should match that in the lists of states and actions. 

**Note**: Don't forget to import `numpy`.

---

In [41]:
import numpy as np
states = ['Ac', 'Ad']
actions = ['peek', 'guessAc', 'guessAd']
observations = ['seeAc', 'seeAd', 'seeNothing']

Ppeek = np.identity(2)
PAc = np.ones((2,2)) * 0.5
PAd = PAc

P = np.zeros((3,2,2))
P[0] = Ppeek
P[1] = PAc
P[2] = PAd

Opeek = np.zeros((2,3))
Opeek[:, 0] = [0.9, 0.1]
Opeek[:, 1] = [0.1, 0.9]
OAc = np.zeros((2,3))
OAc[:, 2] = 1
OAd = OAc

O = np.zeros((3,2,3))
O[0] = Opeek
O[1] = OAc
O[2] = OAd

c = np.ones((2,3))
c[:,0] = [0.5, 0.5]
c[:,1] = [0, 1]
c[:,2] = [1, 0]

print('Ppeek\n', Ppeek)
print('PAc\n', PAc)
print('PAd\n', PAd)
print('OPeek\n', Opeek)
print('OAc\n', OAc)
print('OAd\n', OAd)
print('C\n', C)

Ppeek
 [[ 1.  0.]
 [ 0.  1.]]
PAc
 [[ 0.5  0.5]
 [ 0.5  0.5]]
PAd
 [[ 0.5  0.5]
 [ 0.5  0.5]]
OPeek
 [[ 0.9  0.1  0. ]
 [ 0.1  0.9  0. ]]
OAc
 [[ 0.  0.  1.]
 [ 0.  0.  1.]]
OAd
 [[ 0.  0.  1.]
 [ 0.  0.  1.]]
C
 [[ 0.5  0.   1. ]
 [ 0.5  1.   0. ]]


### 2. Sampling

You are now going to sample random trajectories of your POMDP and observe the impact it has on the corresponding belief.

---

#### Activity 2.

Generate a random POMDP trajectory using a uniformly random policy. In particular, from a random initial state $x_0$ generate:

1. A sequence of 10,000 states by selecting the actions uniformly at random;
2. The corresponding sequence of 10,000 actions;
3. The corresponding sequence of 10,000 observations.

---

In [32]:

def get_random_trajectory(x0, n=10000):
    indexx = states.index(x0)
    x = np.zeros(2)
    x[indexx] = 1
    trajectory_states = []
    trajectory_actions = []
    trajectory_observations = []
    current_state = x
    for i in range(n):
        action = np.random.choice(len(actions))
        Paction = P[action]
        Oaction = O[action]
        index_state = np.random.choice(len(states), p = current_state.dot(Paction))
        current_state = np.zeros(2)
        current_state[index_state] = 1
        obs = np.random.choice(len(observations), p = current_state.dot(Oaction))
        
        trajectory_states.append(states[current_state.argmax()])
        trajectory_actions.append(actions[action])
        trajectory_observations.append(observations[obs])
    return trajectory_states, trajectory_actions, trajectory_observations
        
rnd_s, rnd_a, rnd_o = get_random_trajectory(states[np.random.choice(len(states))])
    

---

#### Activity 3.

For the POMDP trajectory generated in Activity 2, compute the corresponding sequence of beliefs, assuming that the initial belief is $\mathbf{b}_0=[0.5, 0.5]$. Report the resulting beliefs, ignoring duplicate beliefs or beliefs whose distance is smaller than $10^{-4}$.

**Note 1:** You may want to define a function `belief_update` that receives a belief, an action and an observation and returns the updated belief.

**Note 2:** To compute the distance between vectors, you may find useful `numpy`'s function `linalg.norm`.


---

In [38]:

# b is the belief
# a is the index of the action
# z is the index of the observation
def belief_update(b, a, z):
    res = b.dot(P[a]).dot(np.diag(O[a][:, z]))
    return res/np.sum(res)

current_belief = np.ones(2) * 0.5
beliefs = []
beliefs.append(current_belief)
for i in range(len(rnd_s)):
    current_belief = belief_update(current_belief, actions.index(rnd_a[i]), observations.index(rnd_o[i]))
    is_dup = False
    for k in beliefs:
        if (np.linalg.norm(current_belief - k) < 10e-4):
            is_dup = True
            break
    if (not is_dup):
        beliefs.append(current_belief)
    
print(beliefs)

[array([ 0.5,  0.5]), array([ 0.1,  0.9]), array([ 0.9,  0.1]), array([ 0.98780488,  0.01219512]), array([ 0.99863014,  0.00136986]), array([  9.99847607e-01,   1.52392563e-04]), array([ 0.01219512,  0.98780488]), array([ 0.00136986,  0.99863014]), array([  1.52392563e-04,   9.99847607e-01])]


### 3. Solution methods

In this section you are going to compare different non-exact solution methods.

---

#### Activity 4

Compute the solution for the underlying MDP and report the corresponding optimal policy and optimal cost-to-go. 

** Note:** You may reuse code from previous labs.

---

In [52]:
def evaluate(probs, policy, costs, gama = 0.99):
    # computes transition probabilities for a given policy 
    Ppolicy = np.einsum('nrm,rn->rm', probs, policy)
    # computes cost for a given policy
    Cpolicy = np.sum(costs * policy, axis=1).reshape(-1, 1)
   
    I = np.identity((len(states)))
    # computes discounted cost-to-go 
    return np.dot(np.dot(I, np.linalg.inv(I - gama*Ppolicy)), Cpolicy), Ppolicy, Cpolicy

# computes C + gama * PJ for every action. returns an array of J's for every action
def calculateQ(j, c):
    result = np.zeros(shape=(len(states), len(actions)))
    for i in range(len(actions)):
        result[:, i] = (c[:, i].reshape(-1, 1) + gama * P[i].dot(j)).reshape(-1)
    return result


def policy_iteration(gama, P, c):
    exit = False
    iterations = 0
    policy = np.ones(shape = (len(states), len(actions))) / len(actions)
    while not exit:
        # evaluates policy
        ev, _, _ = evaluate(P, policy, c)
        # computes Q function
        Q = calculateQ(ev, c)
        a = np.isclose(Q, Q.min(axis=1).reshape(-1, 1), atol=1e-10, rtol=1e-10)
        # improves policy
        newpolicy = np.divide(a,np.sum(a, axis=1).reshape(-1, 1))
        exit = (policy == newpolicy).all()
        policy = newpolicy 
        iterations += 1
    return newpolicy

In [54]:
optimal_policy_mdp = policy_iteration(0.99, P, c)
optimal_cost_mdp, _, _ = evaluate(P, optimal_policy, c, 0.99)
print(optimal_policy_mdp)
print(optimal_cost_mdp)

[[ 0.  1.  0.]
 [ 0.  0.  1.]]
[[ 0.]
 [ 0.]]


---

#### Activity 5

For each of the beliefs computed in Activity 3, compute the action prescribed by:

* The MLS heuristic;
* The AV heuristic;
* The Q-MDP heuristic.

---

In [57]:
def mls_heuristic(b):
    return optimal_policy_mdp[b.argmax()].argmax()
    
def avs_heuristic(b):
    return b.dot(optimal_policy_mdp).argmax()

def qmdp_heuristic(b):
    return b.dot(calculateQ(optimal_cost_mdp, c)).argmin()

for belief in beliefs:
    print('belief', belief)
    print('mls', actions[mls_heuristic(belief)])
    print('avs', actions[avs_heuristic(belief)])
    print('qmdp', actions[qmdp_heuristic(belief)])

belief [ 0.5  0.5]
mls guessAc
avs guessAc
qmdp peek
belief [ 0.1  0.9]
mls guessAd
avs guessAd
qmdp guessAd
belief [ 0.9  0.1]
mls guessAc
avs guessAc
qmdp guessAc
belief [ 0.98780488  0.01219512]
mls guessAc
avs guessAc
qmdp guessAc
belief [ 0.99863014  0.00136986]
mls guessAc
avs guessAc
qmdp guessAc
belief [  9.99847607e-01   1.52392563e-04]
mls guessAc
avs guessAc
qmdp guessAc
belief [ 0.01219512  0.98780488]
mls guessAd
avs guessAd
qmdp guessAd
belief [ 0.00136986  0.99863014]
mls guessAd
avs guessAd
qmdp guessAd
belief [  1.52392563e-04   9.99847607e-01]
mls guessAd
avs guessAd
qmdp guessAd


---

#### Activity 6

Suppose that the optimal cost-to-go function for the POMDP can be represented using the $\alpha$-vectors

$$
\left\{
\begin{bmatrix}
2.795\\
3.795
\end{bmatrix},
\begin{bmatrix}
3.795\\
2.795
\end{bmatrix},
\begin{bmatrix}
3.105\\
3.105
\end{bmatrix}\right\}$$

corresponding to the actions 'Guess clubs', 'Guess diamonds' and 'Peek', respectively. Represent the optimal cost-to-go function and compare the optimal policy with the MDP heuristics from Activity 5 in the beliefs computed in Activity 3.

** Note: ** Don't forget to import `matplotlib`, and use the magic `%matplotlib notebook`.

---

In [68]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib
a_clubs = np.array([2.795, 3.795])
a_diamonds = np.array([3.795, 2.795])
a_peek = np.array([3.105, 3.105])
plt.plot(a_clubs)
plt.plot(a_diamonds)
plt.plot(a_peek)
plt.show()

Using matplotlib backend: TkAgg
